### Preamble

The basic idea here is to set up a new [Python](http://www.python.org) project repository and publish it on Github. The procedure would be similar for other lanuguages.

This guide assumes you have a Github account and are working on some flavour of Linux as an operating system (you mileage may vary otherwise).

All of this set up can be done in the terminal of an [Intergrated Development Environment](https://www.simplilearn.com/tutorials/python-tutorial/python-ide) such as [Visual Studio Code](https://code.visualstudio.com/). A guide to setting up VScode is [here](./vscode-guide.md). 

The following setup uses [poetry](https://python-poetry.org/) to manage a virtual environment for the project and [pre-commit](https://pre-commit.com/) to apply coding formatters and linters to the code before it is committed to git - thereby giving the code a consistent style.


### General system-wide setup

You should not install additional packages into the system Python version, rather you should use a separate [virtual enviroment][vit-env] to manage the dependencies of each project. If a package needs to be system-wide (here both git and poetry) we can use [pipx](https://github.com/pypa/pipx).

[Install pipx][pipx-install]:
```bash
    $ sudo python3 -m pip install --user pipx
    $ sudo python3 -m pipx ensurepath
```

Install [git][git]:

```bash
    $ sudo pipx install git
```

Install [poetry][poetry]:

```bash
    $ sudo pipx install poetry
```

Configure the following poetry defaults:
```bash
    $ poetry config virtualenvs.in-project true
    $ poetry config virtualenvs.prompt "(.venv)"
    $ poetry config virtualenvs.path " "
    $ poetry config --list
```
```python
    cache-dir = "///.cache/pypoetry"
    experimental.system-git-client = false
    installer.max-workers = null
    installer.modern-installation = true
    installer.no-binary = null
    installer.parallel = true
    keyring.enabled = true
    solver.lazy-wheel = true
    virtualenvs.create = true
    virtualenvs.in-project = true
    virtualenvs.options.always-copy = false
    virtualenvs.options.no-pip = false
    virtualenvs.options.no-setuptools = false
    virtualenvs.options.system-site-packages = false
    virtualenvs.path = " "
    virtualenvs.prefer-active-python = false
    virtualenvs.prompt = "(.venv)"
    warnings.export = true
```

[vit-env]: https://medium.com/pythoneers/why-do-we-need-a-virtual-environment-for-a-python-project-37a6af754044
[git]: https://git-scm.com/
[poetry]: https://python-poetry.org/
[pipx-install]: https://pipx.pypa.io/stable/installation/

### Setting up a new project

Initialise the project using poetry

```bash
    $ mkdir bioinformatics-development-setup
    $ cd bioinformatics-development-setup
    $ poetry init
```

Add the [black](https://github.com/psf/black) linter and [pre-commit](https://pre-commit.com/) git hooks to the project

```bash
    $ poetry add black --group dev
    $ poetry add pre-commit --group dev
    $ cat pyproject.toml
```

```toml
    [tool.poetry]
    name = "bioinformatics-development-setup"
    version = "0.1.0"
    description = "A basic development environment for bioinformatics"
    authors = ["Cymon J. Cox <cymon.cox@googlemail.com>"]
    license = "MIT"
    readme = "README.ipynb"

    [tool.poetry.dependencies]
    python = "^3.10"

    [tool.poetry.group.dev.dependencies]
    black = "^24.8.0"
    pre-commit = "^3.8.0"

    [build-system]
    requires = ["poetry-core"]
    build-backend = "poetry.core.masonry.api"
```

##### Setup "pre-commit" hooks to linters

Write the following configuration file to the project directory:
```bash
$ cat .pre-commit-config.yaml
```
```yaml
repos:
-   repo: https://github.com/pre-commit/pre-commit-hooks
    rev: v2.3.0
    hooks:
    -   id: check-yaml
    -   id: end-of-file-fixer
    -   id: trailing-whitespace
    -   id: check-json
    -   id: check-added-large-files
    -   id: check-toml
    -   id: check-yaml
    -   id: detect-private-key
-   repo: https://github.com/psf/black
    rev: 22.10.0
    hooks:
    -   id: black    
-   repo: https://github.com/asottile/pyupgrade
    rev: v3.17.0
    hooks:
    -   id: pyupgrade
        args: [--py38-plus]
-   repo: https://github.com/asottile/reorder-python-imports
    rev: v3.13.0
    hooks:
    -   id: reorder-python-imports
        exclude: ^(pre_commit/resources/|testing/resources/python3_hooks_repo/)
        args: [--py39-plus, --add-import, 'from __future__ import annotations']
```

Install the pre-commit configuration:
```bash
$ pre-commit install
pre-commit installed at .git/hooks/pre-commit
```

##### Initialise the project as a git repository

 Write the .gitignore file

```bash
    $ cat .gitignore
    # jupyter
    .ipynb_checkpoints/*

    # poetry
    poetry.lock
    .venv

    # pre-commit config
    .pre-commit-config.yaml

    # VScode
    bioinformatics-development-setup.code-workspace
    .vscode

    #Others
    *.swp
```

Initialise a git repository

```bash
    $ git init
    $ git status
    On branch main

    No commits yet

    Untracked files:
    (use "git add <file>..." to include in what will be committed)
            .gitignore
            README.ipynb
            pyproject.toml
```

Add the files to the repository and commit
```bash
    $ git add .
    $ git commit -m "Initial commit"
    Check Yaml...........................................(no files to check)Skipped
    Fix End of Files.........................................................Failed
    - hook id: end-of-file-fixer
    - exit code: 1
    - files were modified by this hook

    Fixing .gitignore

    Trim Trailing Whitespace.................................................Passed
    Check JSON...............................................................Passed
    Check for added large files..............................................Passed
    Check Toml...............................................................Passed
    Check Yaml...........................................(no files to check)Skipped
    Detect Private Key.......................................................Passed
    black................................................(no files to check)Skipped
    pyupgrade............................................(no files to check)Skipped
    Reorder python imports...............................(no files to check)Skipped
```
As you see the ```.gitignore``` file failed the "Fix End of Files" check. The file was modified but the commit did therefore not take place:
```bash
    $ git status
    On branch main

    No commits yet

    Changes to be committed:
      (use "git rm --cached <file>..." to unstage)
            new file:   .gitignore
            new file:   README.ipynb
            new file:   pyproject.toml

    Changes not staged for commit:
      (use "git add <file>..." to update what will be committed)
      (use "git restore <file>..." to discard changes in working directory)
            modified:   .gitignore
```
Re-stage and re-commit:
```bash
    $ git add .
    $ git commit -m "Initial commit"
    Check Yaml...........................................(no files to check)Skipped
    Fix End of Files.........................................................Passed
    Trim Trailing Whitespace.................................................Passed
    Check JSON...............................................................Passed
    Check for added large files..............................................Passed
    Check Toml...............................................................Passed
    Check Yaml...........................................(no files to check)Skipped
    Detect Private Key.......................................................Passed
    black................................................(no files to check)Skipped
    pyupgrade............................................(no files to check)Skipped
    Reorder python imports...............................(no files to check)Skipped
    [main (root-commit) 86f5179] Initial commit
    3 files changed, 147 insertions(+)
    create mode 100644 .gitignore
    create mode 100644 README.ipynb
    create mode 100644 pyproject.toml
 ```

##### Publishing the project to Github

You first must create the Github repository throught the Github web-interface
- Log in to [Github](https://github.com/) and open the Dashboard

- On the top-left above the Top Repositories listing is a green button `new`
- Give you new repository the same name that you gave the project directory; here `bioinformatics-development-setup`
- Choose a license, add a README.md, but do not add a default ```.gitignore```
- Create repository
- Copy the URL from the browser address bar:`https://github.com/cymon/bioinformatics-development-setup`

Back in your terminal in the project directory, add the URL as the git remote address:
```bash
    $ git remote add origin https://github.com/cymon/bioinformatics-development-setup
    $ git config --list
    user.email=cymon.cox@googlemail.com
    user.name=Cymon J. Cox
    pull.rebase=false
    init.defaultbranch=main
    core.repositoryformatversion=0
    core.filemode=true
    core.bare=false
    core.logallrefupdates=true
    remote.origin.url=https://github.com/cymon/bioinformatics-development-setup
    remote.origin.fetch=+refs/heads/*:refs/remotes/origin/*
```
Then push to Github setting upstream and forcing the update to the remote repository - you should not use force `-f` again, and you could lose commits in Github if you do:
```bash
    $ git push -u -f origin main
```
Thereafter use `$ git pull origin` `$ git push origin`